In [1]:
import altair as alt
from vega_datasets import data

In [5]:
cars = data.cars.url

alt.Chart(cars).mark_point().encode(
    x='Acceleration:Q',
    y='Horsepower:Q'
)

alt.Chart(...)

In [3]:
import os
import json
import pandas as pd
from collections import defaultdict

In [ ]:
## TODO
# - kumulativny graf pre vsetky thresholds a Recall, Accuracy, F1 Score (metrics.json)
# - pregenerovat Epoch_Loss, Accuracy, Detection (AP)

In [10]:
source = data.stocks()

In [13]:
source

,symbol,date,price
0,MSFT,2000-01-01,39.81
1,MSFT,2000-02-01,36.35
2,MSFT,2000-03-01,43.22
3,MSFT,2000-04-01,28.37
4,MSFT,2000-05-01,25.45
...,...,...,...
555,AAPL,2009-11-01,199.91
556,AAPL,2009-12-01,210.73
557,AAPL,2010-01-01,192.06
558,AAPL,2010-02-01,204.62


In [18]:
## Cumulative Chart
# load input data
max_data = defaultdict(lambda: {"recall": 0.0, "precision": 0.0, "f1-score": 0.0})

experiment_folder = '../output_models/cross_subject-batch_1_lr_00005_rs_2021_03_09-16_05_14'
with open(os.path.join(experiment_folder, 'metrics.json'), 'r') as mf:
    for line in mf:
        data = json.loads(line.rstrip('\n'))['thresholds']
        for th, values in data.items():
            max_data[th]['recall'] = max(max_data[th]['recall'], round(values['recall'], 4))
            max_data[th]['precision'] = max(max_data[th]['precision'], round(values['precision'], 4))
            max_data[th]['f1-score'] = max(max_data[th]['f1-score'], round(values['f1-score'], 4))

In [55]:
# generate dataframe
type_column = ['recall'] * 9 + ['precision'] * 9 + ['f1-score'] * 9
th_column = list(max_data.keys()) * 3

value_column = [v['recall'] for _, v in max_data.items()]
value_column += [v['precision'] for _, v in max_data.items()]
value_column += [v['f1-score'] for _, v in max_data.items()]

th_df = pd.DataFrame({'metric': type_column, 'threshold': th_column, 'value': value_column})
th_df

,metric,threshold,value
0,recall,0.1,0.2156
1,recall,0.2,0.1626
2,recall,0.3,0.1367
3,recall,0.4,0.1176
4,recall,0.5,0.1030
5,recall,0.6,0.0900
6,recall,0.7,0.0769
7,recall,0.8,0.0610
8,recall,0.9,0.0423
9,precision,0.1,0.1516


In [56]:
# generate chart
chart = alt.Chart(th_df).mark_line().encode(
    x='threshold',
    y='value',
    color='metric',
    #strokeDash='metric',
)
chart.configure_view(
    height=250,
    width=350
)

alt.Chart(...)

In [67]:
folder = '../data/board-data/'

In [83]:
def load_file(file_name: str):
    return pd.read_csv(os.path.join(folder, file_name)), file_name.split('-')[0]    

In [128]:
## Generate epoch-loss chart
file_name = 'cv_oe_lr_0005-epoch_loss.csv'
df, char_name = load_file(file_name)

b = alt.Chart(df).mark_line(interpolate='linear', size=2, opacity=0.4, color='#FF7F50').encode(
    x=alt.X('Step', axis=alt.Axis(title='Epoch')),
    y=alt.Y('Value:Q', scale=alt.Scale(domain=(0.02, 0.12)))
).properties(
    title=f'{char_name} Epoch-Loss'
)
b = b + b.transform_loess('Step', 'Value').mark_line(color='#FF7F50')

b.save(os.path.join(folder, f'{char_name.lower()}-epoch_loss.png'))

b

alt.LayerChart(...)

In [140]:
## Generate accuracy chart
file_name = 'cv_oe_lr_0005_batch_4-accuracy.csv'
df, char_name = load_file(file_name)

b = alt.Chart(df).mark_line(interpolate='linear', size=2, opacity=0.4, color='#FF7F50').encode(
    x=alt.X('Step', axis=alt.Axis(title='Epoch')),
    #y=alt.Y('Value:Q', scale=alt.Scale(zero=False))
    y=alt.Y('Value:Q', scale=alt.Scale(domain=(0, 1.0)))
).properties(
    title=f'{char_name} Accuracy'
)
b = b + b.transform_loess('Step', 'Value').mark_line(color='#FF7F50')

b.save(os.path.join(folder, f'{char_name.lower()}-accuracy.png'))

b

alt.LayerChart(...)